# LLM ModelTest #2
#### *ibm-granite/granite-embedding-278m-multilingual*

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
import os

# API KEY 정보로드
load_dotenv()
GEMINI_API_KEY = os.getenv('API_KEY_GEMINI')

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("dx_project")

LangSmith 추적을 시작합니다.
[프로젝트명]
dx_project


In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS


import pandas as pd
import re
import json
from operator import itemgetter

# 랭체인 환경 설정
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
# from langchain_google_genai import ChatGoogleGenerativeAI
#from langchain_core.prompts import PromptTemplate



In [4]:
### 05. 임베딩 모델 생성
# https://huggingface.co/ibm-granite/granite-embedding-278m-multilingual
embeddings = HuggingFaceEmbeddings(model_name='ibm-granite/granite-embedding-278m-multilingual')

C:\Users\nammi\AppData\Local\Temp\ipykernel_15644\3636801640.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='ibm-granite/granite-embedding-278m-multilingual')
c:\Users\nammi\AppData\Local\pypoetry\Cache\virtualenvs\lgdxteam2-aSyY5fLw-py3.11\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
# Chroma 벡터스토어 로드
vectorstore = Chroma(persist_directory="../data/movie_4000_vectorstore_2", embedding_function=embeddings)
# Fassis 벡터스토어 로드
vectorstore_800 = FAISS.load_local('../data/movies_vectorstore_faiss_800', embeddings = embeddings, allow_dangerous_deserialization=True)
vectorstore_1500 = FAISS.load_local('../data/movies_vectorstore_faiss_1500', embeddings = embeddings, allow_dangerous_deserialization=True)

C:\Users\nammi\AppData\Local\Temp\ipykernel_15644\4161624014.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="../data/movie_4000_vectorstore_2", embedding_function=embeddings)


<br><hr>

In [6]:
# 수정정한 셀
# StructuredOutputParser 사용
response_schemas = [
  ResponseSchema(name="type",
                 description="사용자의 입력을 세 가지 범주('정보검색', '추천요청', '일반대화') 중 하나로 구분")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 출력 지시사항 파싱
format_instructions = output_parser.get_format_instructions()

In [7]:
# 예시 형식을 위에서 정의한 format_instructions로 변경
# from_template 매개변수 추가
# 1. 사용자 입력값 유형 분류용 프롬프트
classification_template = """사용자의 입력을 다음 세 가지 범주 중 하나로 분류하세요:
1️⃣ **"정보검색"**: 특정 영화, 드라마, 배우, 감독, 러닝타임, 개봉 연도, 수상 내역, 필모그래피 등 **사실적인 정보를 찾는 질문**
   - 기대되는 응답 예시: 배우가 출연한 드라마/영화 목록, 특정 연도의 개봉작 리스트 등
2️⃣ **"추천요청"**: 특정 장르, 배우, 테마(예: 좀비, 시간여행), 감성(예: 힐링, 긴장감) 등에 대한 **추천을 요청하는 질문**
   - 기대되는 응답 예시: 특정 조건을 만족하는 영화/드라마 추천
3️⃣ **"일반대화"**: 서비스와 무관한 일반적인 대화 (예: 날씨, AI 관련 질문, 잡담)

#### **예시 형식**
{format_instructions}

<user_input>
{user_input}
</user_input>
"""
classification_prompt = ChatPromptTemplate.from_template(classification_template,
                                                         partial_variables={'format_instructions': format_instructions})


In [8]:
# ------------------------------------------
# 2. LLM을 이용한 질문 유형 분류 체인
classification_chain = (
  classification_prompt #prompt
  | ChatOpenAI(model='gpt-4o-mini', temperature=0)
  # 수정된 부분 strOutputParser()
  | output_parser
)

In [9]:
# 질문 분류 테스트
print(classification_chain.invoke({'user_input': "2023년에 개봉한 액션 영화 뭐 있어?"}))

# 추천 요청 예상 질문
print(classification_chain.invoke({'user_input': '디카프리오가 주연한 영화 추천해줘.'}))

# 일반 대화 예상 질문
print(classification_chain.invoke({'user_input': "너가 제일 좋아하는 영화 뭐야?"}))

{'type': '정보검색'}
{'type': '추천요청'}
{'type': '일반대화'}


In [10]:
# 딕셔너리 자료형을 string으로 변환하는 함수
# keyword 부분 남김
def format_change(classification_result: dict, user_input: str) -> str:
    type_value = classification_result.get("classification_result", {}).get("type", "일반대화")
    # keywords = classification_result.get("classification_result", {}).get("keywords", [])

    # string 자료형으로 변경
    # formatted_str = f"type: '{type_value}', keywords: {keywords}, user_input: '{user_input}'"
    formatted_str = f"type: '{type_value}', user_input: '{user_input}'"
    return formatted_str

<hr>

# chain - default

In [11]:
# default_chain 생성 (사용자의 의미없는 입력값에 대해 정해진 답변을 할 때)
# 프롬프트 템플릿 설정
default_template = """
"You are a chatbot that must always respond with '🐶: 멍멍!'.
No matter what question the user asks, always reply with '🐶: 멍멍!'"

[사용자 입력]:
{user_input}
[분류 결과]:
{classification_result}
"""
default_prompt = ChatPromptTemplate.from_template(default_template)

default_llm = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0)

# langchain 체인 구성
default_chain = (
    {"classification_result": RunnablePassthrough(),
     "user_input": RunnablePassthrough()}
  | default_prompt
  | default_llm 
  | StrOutputParser()
)

<hr>

##  search-chain 

In [28]:
# 검색기 생성
# mmr 중복 피하기, 문서의관련성과 차별성 고려, 
# retriever = vectorstore.as_retriever(
#     search_type="similarity_score_threshold",   
#     search_kwargs={"score_threshold": 0.3}
# )

search_template = """
사용자가 영화나 드라마에 대한 정보를 검색하고 있습니다.
다음 사용자 질문과 관련된 **가장 적절한 문서(컨텐츠)를 벡터스토어에서 검색**한 후, 
아래 JSON 리스트 형식으로 반환하세요.  
JSON 이외의 설명은 하지 마세요.
출력은 `index`, `genre`(장르)와 `title`(영화 제목)은 retrieved_context에서 가져온 메타데이터를 기반으로 합니다.
검색 결과에서 최대 5개 문서에 대해서 출력하세요.

### **예시 응답 형식**
```json
{{
  {{"title": "더 이퀄라이저",
  "genre": "SF", "스릴러"}},

  {{"title": "더 이퀄라이저 2",
  "genre": "SF", "스릴러"}},

  {{"title": "더 이퀄라이저 3",
  "genre": "SF", "스릴러"}}

}}

[사용자 입력과 분류 결과]:
{user_input}

[Context]: 
{retrieved_context} 
[Answer]:

"""
search_prompt = ChatPromptTemplate.from_template(search_template)
# search_chain = (
#     {
#       "user_input": RunnablePassthrough(),
#       "retrieved_context": retriever
#     }
#     | search_prompt
#     | default_llm
#     | StrOutputParser()
#     )
# search_chain.invoke("더 이퀄라이저 시리즈가 몇개야 내놔")

<br><hr>

## 04. Full Chain 연결

In [29]:
def process_user_input(classification_result: dict, user_input: str):
    # 사용자의 입력 유형 분류
    print(classification_result)
    print(user_input)
    type_value = classification_result.get("type", "일반대화")  # 기본값 설정
    # keywords = classification_data.get("keywords", [])  # 기본값 설정
    
    print(f"===================== Type: {type_value}")
    # print(f"===================== Keywords: {keywords}")
    
    if type_value == '정보검색':
        if "영화" in user_input:
            retriever = vectorstore_1500.as_retriever(
                search_type="similarity_score_threshold",   
                search_kwargs={"score_threshold": 0.3}
                
            )
        elif "드라마" in user_input:
            retriever = vectorstore.as_retriever(
                search_type="similarity_score_threshold",
                search_kwargs={"score_threshold": 0.3}
            )
        # 둘 다 없으면 "해당하는 콘텐츠가 없습니다" 반환
        else:
            return "해당하는 콘텐츠가 없습니다."
        search_chain = (
            {
                "user_input": RunnablePassthrough(),
                "retrieved_context": retriever
            }
            | search_prompt
            | default_llm
            | StrOutputParser()
        )

        formatted_string = format_change(classification_result, user_input)
        return search_chain.invoke(formatted_string)
    elif type_value == '추천요청':
        # formatted_string = format_change(classification_result, user_input)
        return "정보검색 체인 실행은 여기!!!"
    # recommend_chain.invoke(formatted_string)
    else:
        return default_chain.invoke({"classification_result": classification_result})

In [30]:
full_chain = (
  {"classification_result": classification_chain,
   "user_input":itemgetter("user_input")} #{"user_input":"더 이퀄라이}
  | RunnableLambda(lambda x: process_user_input(x["classification_result"], x["user_input"]))
  | StrOutputParser()  
)

In [32]:
full_chain.invoke({"user_input":"더 이퀄라이저 영화 시리즈가 몇개야 내놔"})

{'type': '정보검색'}
더 이퀄라이저 영화 시리즈가 몇개야 내놔
===================== Type: 정보검색


'```json\n[\n  {\n    "index": 1,\n    "title": "더 이퀄라이저",\n    "genre": "액션, 범죄, 드라마"\n  },\n  {\n    "index": 2,\n    "title": "더 이퀄라이저 2",\n    "genre": "액션, 범죄, 드라마"\n  },\n  {\n    "index": 3,\n    "title": "더 이퀄라이저 3",\n    "genre": "액션, 범죄, 드라마"\n  }\n]\n```'

In [28]:
pd.read_csv("C:/Users/USER/Downloads/영화_TMDB_5800_Mapping-최종(250107).csv")

,name,orgnl_cntry,movie_id,overview,release_date,adult,backdrop_path,original_language,original_title,poster_path,popularity,runtime,vote_average,vote_count,genre,test
0,노크 노크,미국,263472,가족에게 헌신적이고 누구보다 아내와 아이들을 사랑하는 성공한 건축가 에반(키아누 리...,2015-06-26,False,/nJkp7hYKScmvQuqfptIIclFxZ94.jpg,en,Knock Knock,/8hjIz6BkZiBSeuHYs1e8eO0w489.jpg,61.593,99,5.368,3203,"공포, 스릴러",1
1,노크: 더 하우스,미국,709631,"이제 막 여덟 살이 된 소년 ‘피터’ . 밤이 찾아오면, 방에서 의문의 노크 소리가...",2023-07-19,False,/nYDPmxvl0if5vHBBp7pDYGkTFc7.jpg,en,Cobweb,/5Ol63pZT8JfYgFMfADqySpgwkdj.jpg,36.104,88,6.509,876,공포,1
2,노크,대한민국,255250,미대생 정화(서우)는 학교 근처의 작은 아파트로 이사를 하던중 순간적으로 발을 헛디...,2012-11-29,False,/4etMq6yuYyzzARECneGst6l1RWr.jpg,ko,노크,/bGgrseFG2FZd7dIhBmsFupEhlyt.jpg,0.259,100,3.700,3,"공포, TV 영화, 로맨스",2
3,한밤의 황당한 저주,미국,398798,야간 경비로 첫 출근을 하게 된 저스틴은 선임들의 고약한 인터뷰를 무사히 마치고 전...,2017-03-04,False,/r0HMSRMOCSXZSj6fcN7CXdJLtSk.jpg,en,The Night Watchmen,/b0kxlzG2mkJ3ljWOEtZ95Knpw4y.jpg,7.981,90,5.100,126,"코미디, 공포",1
4,신세계,대한민국,165213,경찰청 수사기획과 강과장은 국내 최대 범죄 조직 골드문이 기업형 조직으로 세력이 확...,2013-02-21,False,/8OceZqoCwVoc3JMN7ffsaj4BGFM.jpg,ko,신세계,/yiDQaLGF6hXVdiKodfdurUcgxWV.jpg,16.436,134,7.400,461,"스릴러, 범죄, 드라마",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5795,샤프트,미국,479,돈 많은 아버지를 둔 월터 웨이드(크리스찬 베일)는 사소한 시비 끝에 젊은 흑인 학...,2000-06-15,False,/sblTwYXpm6Nrl4qewg3526lBdlK.jpg,en,Shaft,/5ncvnNPaN73cOqD4muE413sSOyY.jpg,22.547,99,6.019,1273,"액션, 모험, 범죄, 스릴러",1
5796,페이첵,미국,9620,"가까운 미래, 천재 공학자 마이클 제닝스는 각종 회사의 일급 프로젝트를 수행하고 있...",2003-12-25,False,/rObU664nB6WmWUduCcwYOUYkPAt.jpg,en,Paycheck,/a9DUh3KJlxTHgngtVcCwPAfgknS.jpg,16.652,119,6.200,1808,"액션, 모험, 미스터리, SF, 스릴러",1
5797,아미스타드,미국,11831,"칠흑 같이 어두운 밤, 흑인들의 지도자격인 신케이는 쇠고랑을 푸는 데 성공하고 아미...",1997-12-10,False,/2lI5rNJeYHShg9okpEXSFzQZqWj.jpg,en,Amistad,/6QqNyIHKow0jngiQgTNBOBrLILM.jpg,4.085,155,7.026,1292,"드라마, 역사, 미스터리",1
5798,매니악,미국,27346,프랭크(조 스피넬 분)는 몇 년전 교통사고로 죽은 어머니를 항상 그리워한다. 매춘부...,1980-05-10,False,/h5X5zO40G0pjPkyyzGh9PBsPI0B.jpg,en,Maniac,/aKIBqbUZAEwOML4oMMjZS7m4Gvj.jpg,12.009,88,6.457,440,공포,1
